## Import Library

In [1]:
import cv2
import numpy as np
import tensorflow as tf
import copy
import itertools
import mediapipe as mp

## Define Pre-Requisite

### Load Model and class target

In [3]:
#Hands detector
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

#Model Classifier
model = tf.keras.models.load_model("models/bestModel_v2")

In [4]:
#Define target class
class_names = {
    0:"close",
    1:"open",
    2:"slide left",
    3:"slide right"
}

### Function to PreProcess and Annotate image

In [5]:
#get coordinate of bounding box
def calc_bounding_rect(image, landmarks): 
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv2.boundingRect(landmark_array)

    return [x, y, x + w, y + h]

#calculate coordinate of each hand keypoints respective to image pixels
def calc_landmark_list(image, landmarks): 
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point


#calculate coordinate of each hand keypoints respective to bounding box and normalize result
def pre_process_landmark(landmark_list): 
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

### Function for drawing box

In [6]:
#funtion to draw bounding box
def draw_bounding_rect(use_brect, image, brect, text):
    if use_brect:
        # Outer rectangle
        cv2.rectangle(image, (brect[0], brect[1]), (brect[2], brect[3]),
                     (242, 147, 244), 2)
    
        cv2.rectangle(image, (brect[0], brect[1]), (brect[2], brect[1] - 22),
                 (242, 147, 244), -1)
        cv2.putText(image, str(text), (brect[0] + 5, brect[1] - 4),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

    return image



## Real Time Detector

In [7]:
#build camera
cam = cv2.VideoCapture(0)

#start realtime inference
with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5 ) as hands:

    while cam.isOpened():
        #read frames
        ret, frame = cam.read()
        frame.flags.writeable = False

        image = cv2.flip((cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)),1) 
        results = hands.process(image)

        annotated_image = image.copy()

        if results.multi_hand_landmarks:
            
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks,results.multi_handedness):

                #find out box coordinate
                boundary = calc_bounding_rect(image, hand_landmarks)

                #calculate keypoints coordinate relative 0 keypoints(bottom of palm)
                landmark_list = calc_landmark_list(image,hand_landmarks)
                relative_landmark = pre_process_landmark(landmark_list)

                #prediction the target
                y = model.predict(np.expand_dims(relative_landmark, axis=0))
                label = class_names[np.argmax(y)]

            
                #draw box coordinate
                boundary = draw_bounding_rect(True , annotated_image, boundary, label)

                mp_drawing.draw_landmarks(
                        annotated_image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())
                

        #show the frames into the screen
        annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)
        cv2.imshow("Hand Pose Detection", annotated_image)

        #Break for stop the webcam process
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break   
    
    cam.release()
    cv2.destroyAllWindows()